In [ ]:
import cv2
import pytesseract
import numpy as np
import torch
import torchaudio
import moviepy
from transformers import pipeline

# تحميل نموذج التعرف على الكلام من OpenAI Whisper
whisper_model = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# تحميل الفيديو
video_path = "AutoRef-video.mp4"
cap = cv2.VideoCapture(video_path)

# إعداد Tesseract OCR لاستخراج النصوص (نتيجة المباراة)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # تعديل حسب نظامك

# دالة لاستخراج النصوص من إطار معين (مثل النتيجة)
def extract_score(frame):
    roi = frame[50:100, 400:600]  # تعديل حسب موقع النتيجة في الفيديو
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    score_text = pytesseract.image_to_string(gray)
    return score_text.strip()

# تحليل الفيديو إطارًا بإطار
frame_count = 0
previous_score = ""

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % 30 == 0:  # تحليل كل ثانية (30 إطار في الثانية)
        score = extract_score(frame)
        if score and score != previous_score:
            print(f"⚽ تغيير في النتيجة: {score}")
            previous_score = score

cap.release()

# استخراج الصوت من الفيديو
audio_path = "match_audio.wav"
audio = AudioFileClip(video_path)
audio.write_audiofile(audio_path)

# تحويل الصوت إلى نص باستخدام Whisper
def transcribe_audio(audio_path):
    audio_input, _ = torchaudio.load(audio_path)
    transcription = whisper_model(audio_input.numpy())
    return transcription['text']

# استخراج التعليق الصوتي
commentary_text = transcribe_audio(audio_path)
print("🎙️ التعليق الرياضي:")
print(commentary_text)

# تحليل المشاهد الحماسية بناءً على مستوى الصوت
import librosa

def detect_exciting_moments(audio_path, threshold=0.1):
    y, sr = librosa.load(audio_path)
    energy = librosa.feature.rms(y=y)
    exciting_moments = np.where(energy > threshold)[1]  # اللحظات ذات الصوت العالي
    return exciting_moments

exciting_moments = detect_exciting_moments(audio_path)
print(f"🔥 عدد اللحظات الحماسية المكتشفة: {len(exciting_moments)}")

In [ ]:
import cv2
import pytesseract
import numpy as np
import torch
import torchaudio
import moviepy
from transformers import pipeline

# تحميل نموذج التعرف على الكلام من OpenAI Whisper
whisper_model = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# تحميل الفيديو
video_path = "AutoRef-video.mp4"
cap = cv2.VideoCapture(video_path)

# إعداد Tesseract OCR لاستخراج النصوص (نتيجة المباراة)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # تعديل حسب نظامك

# دالة لاستخراج النصوص من إطار معين (مثل النتيجة)
def extract_score(frame):
    roi = frame[50:100, 400:600]  # تعديل حسب موقع النتيجة في الفيديو
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    score_text = pytesseract.image_to_string(gray)
    return score_text.strip()

# تحليل الفيديو إطارًا بإطار
frame_count = 0
previous_score = ""

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % 30 == 0:  # تحليل كل ثانية (30 إطار في الثانية)
        score = extract_score(frame)
        if score and score != previous_score:
            print(f"⚽ تغيير في النتيجة: {score}")
            previous_score = score

cap.release()

# استخراج الصوت من الفيديو
audio_path = "match_audio.wav"
audio = AudioFileClip(video_path)
audio.write_audiofile(audio_path)

# تحويل الصوت إلى نص باستخدام Whisper
def transcribe_audio(audio_path):
    audio_input, _ = torchaudio.load(audio_path)
    transcription = whisper_model(audio_input.numpy())
    return transcription['text']

# استخراج التعليق الصوتي
commentary_text = transcribe_audio(audio_path)
print("🎙️ التعليق الرياضي:")
print(commentary_text)

# تحليل المشاهد الحماسية بناءً على مستوى الصوت
import librosa

def detect_exciting_moments(audio_path, threshold=0.1):
    y, sr = librosa.load(audio_path)
    energy = librosa.feature.rms(y=y)
    exciting_moments = np.where(energy > threshold)[1]  # اللحظات ذات الصوت العالي
    return exciting_moments

exciting_moments = detect_exciting_moments(audio_path)
print(f"🔥 عدد اللحظات الحماسية المكتشفة: {len(exciting_moments)}")

In [5]:
import cv2
import pytesseract
import numpy as np

net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")  
layer_names = net.getLayerNames()  
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

cap = cv2.VideoCapture("AutoRef-video.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width, channels = frame.shape

    # تحويل الصورة إلى بيانات لـ YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    detections = net.forward(output_layers)

    # تحليل النتائج
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # حد الثقة
                center_x, center_y, w, h = map(int, obj[:4] * [width, height, width, height])
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                if class_id == 32:  # كرة القدم في COCO dataset
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    cv2.putText(frame, "Ball", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # استخراج النصوص من الفيديو (مثل النتيجة والوقت)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray, config='--psm 6')
    print("Extracted Text:", text)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov4.cfg in function 'cv::dnn::dnn4_v20240521::readNetFromDarknet'


In [5]:
import whisper
import moviepy
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import arabic_reshaper
from bidi.algorithm import get_display

# تحميل مكتبات NLP الخاصة باللغة العربية
nltk.download('punkt')
nltk.download('stopwords')

# قائمة بالأحداث المهمة في كرة القدم
important_events = {
    "هدف": ["يسجل", "أحرز", "هدف", "هز الشباك"],
    "بطاقة صفراء": ["بطاقة صفراء", "إنذار"],
    "بطاقة حمراء": ["بطاقة حمراء", "طرد"],
    "تبديل": ["تبديل", "استبدال"],
    "فرصة ضائعة": ["أضاع فرصة", "كرة خطيرة", "فرصة ضائعة"]
}

# العبارات الشائعة في التعليق الرياضي
common_phrases = [
    "يا له من هدف!", "كرة خطيرة", "فرصة رائعة", "ضغط متواصل", "المباراة تشتعل"
]

# 🔹 اسم ملف الفيديو المطلوب
video_filename = "AutoRef-video.mp4"

def extract_audio_from_video(video_filename):
    """ استخراج الصوت من الفيديو وحفظه كملف صوتي """
    audio_path = "extracted_audio.wav"
    video = mp.VideoFileClip(video_filename)
    video.audio.write_audiofile(audio_path)
    return audio_path

def transcribe_audio(audio_path):
    """ تحويل الصوت إلى نص باستخدام Whisper """
    model = whisper.load_model("base")  # يمكنك استخدام "small" أو "medium" لدقة أعلى
    result = model.transcribe(audio_path, language="ar")
    return result["text"]

def extract_players(text):
    """ استخراج أسماء اللاعبين من النص تلقائيًا باستخدام تحليل الكلمات """
    words = word_tokenize(text)  # تقسيم النص إلى كلمات
    stop_words = set(stopwords.words("arabic"))  # قائمة كلمات التوقف
    
    # استبعاد الكلمات الشائعة والاحتفاظ بالكلمات التي تبدو كأسماء
    potential_names = [word for word in words if word not in stop_words and len(word) > 2] 
    
    # ترجيح الكلمات الأكثر تكرارًا والتي تذكر بجانب أحداث كرة القدم
    player_names = {}
    for word in potential_names:
        if word in player_names:
            player_names[word] += 1
        else:
            player_names[word] = 1

    # استخراج الأسماء الأكثر تكرارًا والتي من المرجح أن تكون أسماء لاعبين
    sorted_players = sorted(player_names.items(), key=lambda x: x[1], reverse=True)
    top_players = [player[0] for player in sorted_players[:5]]  # اختيار أكثر 5 أسماء تكرارًا
    
    return top_players

def extract_events(text):
    """ استخراج الأحداث المهمة من النص """
    return [event for event, keywords in important_events.items() if any(keyword in text for keyword in keywords)]

def extract_common_phrases(text):
    """ استخراج العبارات الشائعة من النص """
    return [phrase for phrase in common_phrases if phrase in text]

# 🔹 التأكد من وجود ملف الفيديو قبل المتابعة
if os.path.exists(video_filename):
    print(f"🎥 تم العثور على الفيديو: {video_filename}")
    
    # 🔹 استخراج الصوت وتحويله إلى نص
    audio_path = extract_audio_from_video(video_filename)
    transcribed_text = transcribe_audio(audio_path)

    # 🔹 استخراج المعلومات المطلوبة تلقائيًا
    players_mentioned = extract_players(transcribed_text)
    events_detected = extract_events(transcribed_text)
    common_phrases_detected = extract_common_phrases(transcribed_text)

    # 🔹 طباعة النتائج بتنسيق جيد يدعم اللغة العربية
    print("\n==== 📊 تحليل التعليق الرياضي ====\n")
    print("📝 النص المستخرج:\n")
    print(get_display(arabic_reshaper.reshape(transcribed_text)))
    print("\n🎯 أسماء اللاعبين المكتشفة تلقائيًا:", players_mentioned)
    print("\n⚽ الأحداث المهمة المكتشفة:", events_detected)
    print("\n📢 العبارات الشائعة المكتشفة:", common_phrases_detected)

else:
    print(f"❌ لم يتم العثور على ملف الفيديو: {video_filename}! يرجى التأكد من وضعه في نفس المجلد.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


🎥 تم العثور على الفيديو: AutoRef-video.mp4


NameError: name 'mp' is not defined

In [12]:
import whisper
import ffmpeg
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import arabic_reshaper
from bidi.algorithm import get_display

# تحميل مكتبات NLP الخاصة باللغة العربية
nltk.download('punkt')
nltk.download('stopwords')

# قائمة بالأحداث المهمة في كرة القدم
important_events = {
    "هدف": ["يسجل", "أحرز", "هدف", "هز الشباك"],
    "بطاقة صفراء": ["بطاقة صفراء", "إنذار"],
    "بطاقة حمراء": ["بطاقة حمراء", "طرد"],
    "تبديل": ["تبديل", "استبدال"],
    "فرصة ضائعة": ["أضاع فرصة", "كرة خطيرة", "فرصة ضائعة"]
}

# العبارات الشائعة في التعليق الرياضي
common_phrases = [
    "يا له من هدف!", "كرة خطيرة", "فرصة رائعة", "ضغط متواصل", "المباراة تشتعل"
]

# 🔹 اسم ملف الفيديو المطلوب
video_filename = "AutoRef-video.mp4"

def extract_audio_from_video(video_filename):
    """ استخراج الصوت من الفيديو وحفظه كملف صوتي باستخدام FFmpeg """
    audio_path = "extracted_audio.wav"
    ffmpeg.input(video_filename).output(audio_path).run(overwrite_output=True)
    return audio_path

def transcribe_audio(audio_path):
    """ تحويل الصوت إلى نص باستخدام Whisper """
    model = whisper.load_model("base")  # يمكنك استخدام "small" أو "medium" لدقة أعلى
    result = model.transcribe(audio_path, language="ar")
    return result["text"]

def extract_players(text):
    """ استخراج أسماء اللاعبين من النص تلقائيًا باستخدام تحليل الكلمات """
    words = word_tokenize(text)  # تقسيم النص إلى كلمات
    stop_words = set(stopwords.words("arabic"))  # قائمة كلمات التوقف
    
    # استبعاد الكلمات الشائعة والاحتفاظ بالكلمات التي تبدو كأسماء
    potential_names = [word for word in words if word not in stop_words and len(word) > 2] 
    
    # ترجيح الكلمات الأكثر تكرارًا والتي تذكر بجانب أحداث كرة القدم
    player_names = {}
    for word in potential_names:
        if word in player_names:
            player_names[word] += 1
        else:
            player_names[word] = 1

    # استخراج الأسماء الأكثر تكرارًا والتي من المرجح أن تكون أسماء لاعبين
    sorted_players = sorted(player_names.items(), key=lambda x: x[1], reverse=True)
    top_players = [player[0] for player in sorted_players[:5]]  # اختيار أكثر 5 أسماء تكرارًا
    
    return top_players

def extract_events(text):
    """ استخراج الأحداث المهمة من النص """
    return [event for event, keywords in important_events.items() if any(keyword in text for keyword in keywords)]

def extract_common_phrases(text):
    """ استخراج العبارات الشائعة من النص """
    return [phrase for phrase in common_phrases if phrase in text]

# 🔹 التأكد من وجود ملف الفيديو قبل المتابعة
if os.path.exists(video_filename):
    print(f"🎥 تم العثور على الفيديو: {video_filename}")
    
    # 🔹 استخراج الصوت وتحويله إلى نص
    audio_path = extract_audio_from_video(video_filename)
    transcribed_text = transcribe_audio(audio_path)

    # 🔹 استخراج المعلومات المطلوبة تلقائيًا
    players_mentioned = extract_players(transcribed_text)
    events_detected = extract_events(transcribed_text)
    common_phrases_detected = extract_common_phrases(transcribed_text)

    # 🔹 طباعة النتائج بتنسيق جيد يدعم اللغة العربية
    print("\n==== 📊 تحليل التعليق الرياضي ====\n")
    print("📝 النص المستخرج:\n")
    print(get_display(arabic_reshaper.reshape(transcribed_text)))
    print("\n🎯 أسماء اللاعبين المكتشفة تلقائيًا:", players_mentioned)
    print("\n⚽ الأحداث المهمة المكتشفة:", events_detected)
    print("\n📢 العبارات الشائعة المكتشفة:", common_phrases_detected)

else:
    print(f"❌ لم يتم العثور على ملف الفيديو: {video_filename}! يرجى التأكد من وضعه في نفس المجلد.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


🎥 تم العثور على الفيديو: AutoRef-video.mp4


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [13]:
import whisper
from pydub import AudioSegment
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import arabic_reshaper
from bidi.algorithm import get_display

# تحميل مكتبات NLP الخاصة باللغة العربية
nltk.download('punkt')
nltk.download('stopwords')

# قائمة بالأحداث المهمة في كرة القدم
important_events = {
    "هدف": ["يسجل", "أحرز", "هدف", "هز الشباك"],
    "بطاقة صفراء": ["بطاقة صفراء", "إنذار"],
    "بطاقة حمراء": ["بطاقة حمراء", "طرد"],
    "تبديل": ["تبديل", "استبدال"],
    "فرصة ضائعة": ["أضاع فرصة", "كرة خطيرة", "فرصة ضائعة"]
}

# العبارات الشائعة في التعليق الرياضي
common_phrases = [
    "يا له من هدف!", "كرة خطيرة", "فرصة رائعة", "ضغط متواصل", "المباراة تشتعل"
]

# 🔹 اسم ملف الفيديو المطلوب
video_filename = "AutoRef-video.mp4"


def extract_audio_from_video(video_filename):
    """ استخراج الصوت من الفيديو وحفظه كملف صوتي باستخدام Pydub """
    audio_path = "extracted_audio.wav"
    video = AudioSegment.from_file(video_filename)
    video.export(audio_path, format="wav")
    return audio_path


def transcribe_audio(audio_path):
    """ تحويل الصوت إلى نص باستخدام Whisper """
    model = whisper.load_model("base")  # يمكنك استخدام "small" أو "medium" لدقة أعلى
    result = model.transcribe(audio_path, language="ar")
    return result["text"]


def extract_players(text):
    """ استخراج أسماء اللاعبين من النص تلقائيًا باستخدام تحليل الكلمات """
    words = word_tokenize(text)  # تقسيم النص إلى كلمات
    stop_words = set(stopwords.words("arabic"))  # قائمة كلمات التوقف

    # استبعاد الكلمات الشائعة والاحتفاظ بالكلمات التي تبدو كأسماء
    potential_names = [word for word in words if word not in stop_words and len(word) > 2]

    # ترجيح الكلمات الأكثر تكرارًا والتي تذكر بجانب أحداث كرة القدم
    player_names = {}
    for word in potential_names:
        if word in player_names:
            player_names[word] += 1
        else:
            player_names[word] = 1

    # استخراج الأسماء الأكثر تكرارًا والتي من المرجح أن تكون أسماء لاعبين
    sorted_players = sorted(player_names.items(), key=lambda x: x[1], reverse=True)
    top_players = [player[0] for player in sorted_players[:5]]  # اختيار أكثر 5 أسماء تكرارًا

    return top_players


def extract_events(text):
    """ استخراج الأحداث المهمة من النص """
    return [event for event, keywords in important_events.items() if any(keyword in text for keyword in keywords)]


def extract_common_phrases(text):
    """ استخراج العبارات الشائعة من النص """
    return [phrase for phrase in common_phrases if phrase in text]


# 🔹 التأكد من وجود ملف الفيديو قبل المتابعة
if os.path.exists(video_filename):
    print(f"🎥 تم العثور على الفيديو: {video_filename}")

    # 🔹 استخراج الصوت وتحويله إلى نص
    audio_path = extract_audio_from_video(video_filename)
    transcribed_text = transcribe_audio(audio_path)

    # 🔹 استخراج المعلومات المطلوبة تلقائيًا
    players_mentioned = extract_players(transcribed_text)
    events_detected = extract_events(transcribed_text)
    common_phrases_detected = extract_common_phrases(transcribed_text)

    # 🔹 طباعة النتائج بتنسيق جيد يدعم اللغة العربية
    print("\n==== 📊 تحليل التعليق الرياضي ====\n")
    print("📝 النص المستخرج:\n")
    print(get_display(arabic_reshaper.reshape(transcribed_text)))
    print("\n🎯 أسماء اللاعبين المكتشفة تلقائيًا:", players_mentioned)
    print("\n⚽ الأحداث المهمة المكتشفة:", events_detected)
    print("\n📢 العبارات الشائعة المكتشفة:", common_phrases_detected)

else:
    print(f"❌ لم يتم العثور على ملف الفيديو: {video_filename}! يرجى التأكد من وضعه في نفس المجلد.")

C:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


🎥 تم العثور على الفيديو: AutoRef-video.mp4


C:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
C:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
C:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
C:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [3]:
from moviepy
print("تم استيرادMoviepyبنجاح ")

SyntaxError: invalid syntax (4041957353.py, line 1)

In [2]:
import moviepy
print("تم استيرادMoviepyبنجاح ")

تم استيرادMoviepyبنجاح 
